# ITMAL Exercise

REVISIONS||
---------||
2018-1219| CEF, initial.                  
2018-0206| CEF, updated and spell checked. 
2018-0208| CEF, minor text update.
2018-0305| CEF, updated with SHN comments.
2019-0902| CEF, updated for ITMAL v2.
2019-0904| CEF, updated and added conclusion Q.

## Implementing a dummy classifier with fit-predict interface

We begin with the MNIST data-set and will reuse the data loader from Scikit-learn. Next we create a dummy classifier, and compare the results of the SGD and dummy classifiers using the MNIST data...

#### Qb  Load and display the MNIST data

There is a `sklearn.datasets.fetch_openml` dataloader interface in Scikit-learn. You can load MNIST data like 

```python
from sklearn.datasets import fetch_openml
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784',??) # needs to return X, y, replace '??' with suitable parameters! 
# Convert at scale (not always needed)
#X = X / 255.
```

but you need to set parameters like `return_X_y` and `cache` if the default values are not suitable! 

Check out the documentation for the `fetch_openml` MNIST loader, try it out by loading a (X,y) MNIST data set, and plot a single digit via the `MNIST_PlotDigit` function here (input data is a 28x28 NMIST subimage)

```python
%matplotlib inline
def MNIST_PlotDigit(data):
    import matplotlib
    import matplotlib.pyplot as plt
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = matplotlib.cm.binary, interpolation="nearest")
    plt.axis("off")
```

Finally, put the MNIST loader into a single function called `MNIST_GetDataSet()` so you can resuse it later.

In [ ]:
from sklearn.datasets import fetch_openml
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784',return_X_y=True)
# Convert at scale (not always needed)
X = X / 255.

def MNIST_PlotDigit(data):
    import matplotlib
    import matplotlib.pyplot as plt
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = matplotlib.cm.binary, interpolation="nearest")
    plt.axis("off")
    
MNIST_PlotDigit(X[0])    
    
def MNIST_GetDataSet(X, y):
    X, y = fetch_openml('mnist_784',return_X_y=True)
    X = X / 255.
    
print(X.shape)
print(y.shape)

#### Qb  Add a Stochastic Gradient Decent [SGD] Classifier

Create a train-test data-set for MNIST and then add the `SGDClassifier` as done in [HOLM], p82.

Split your data and run the fit-predict for the classifier using the MNIST data.(We will be looking at cross-validation instead of the simple fit-predict in a later exercise.)

Notice that you have to reshape the MNIST X-data to be able to use the classifier. It may be a 3D array, consisting of 70000 (28 x 28) images, or just a 2D array consisting of 70000 elements of size 784.

A simple `reshape()` could fix this on-the-fly:
```python
X, y = MNIST_GetDataSet()

print(f"X.shape={X.shape}") # print X.shape= (70000, 28, 28)
if X.ndim==3:
    print("reshaping X..")
    assert y.ndim==1
    X = X.reshape((X.shape[0],X.shape[1]*X.shape[2]))
assert X.ndim==2
print(f"X.shape={X.shape}") # X.shape= (70000, 784)
```

Remember to use the category-5 y inputs

```python
y_train_5 = (y_train == '5')    
y_test_5  = (y_test == '5')
```
instead of the `y`'s you are getting out of the dataloader...

Test your model on using the test data, and try to plot numbers that have been categorized correctly. Then also find and plots some misclassified numbers.

In [ ]:
from sklearn.linear_model import SGDClassifier
import numpy as np

#Creating the training and test datasets.
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

#Shuffle dataset
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

#Creating the binary classifier for 5!
y_train_5 = (y_train == '5')    
y_test_5  = (y_test == '5')

#Fit dataset
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)
y_pred = sgd_clf.predict(X_test)

print('Accuracy score:', accuracy_score(y_test_5, y_pred))

In [ ]:
#Predict and print a 5
pred5 = sgd_clf.predict([X_test[15]])
print('Prediction for X_test[15]: ',pred5)
MNIST_PlotDigit(X_test[15])    

In [ ]:
#Predict and print a non 5
pred9 = sgd_clf.predict([X_test[16]])
print('Prediction for X_test[15]: ',pred9) 
MNIST_PlotDigit(X_test[16])  

#### Qc Implement a dummy binary classifier

Now we will try to create a Scikit-learn compatible estimator implemented via a python class. Follow the code found in [HOML], p84, but name you estimator `DummyClassifier` instead of `Never5Classifyer`.

Here our Python class knowledge comes into play. The estimator class hierarchy looks like

<img src="https://itundervisning.ase.au.dk/E19_itmal/L02/Figs/class_base_estimator.png" style="width:500px">

All Scikit-learn classifiers inherit from `BaseEstimator` (and possibly also `ClassifierMixin`), and they must have a `fit-predict` function pair (strangely not in the base class!) and you can actually find the `sklearn.base.BaseEstimator` and `sklearn.base.ClassifierMixin` python source code somewhere in you anaconda install dir, if you should have the nerves to go to such interesting details.

But surprisingly you may just want to implement a class that contains the `fit-predict` functions, ___without inheriting___ from the `BaseEstimator`, things still work due to the pythonic 'duck-typing': you just need to have the class implement the needed interfaces, obviously `fit()` and `predict()` but also the more obscure `get_params()` etc....then the class 'looks like' a `BaseEstimator`...and if it looks like an estimator, it _is_ an estimator (aka. duck typing).

Templates in C++ also allow the language to use compile-time duck typing!

> https://en.wikipedia.org/wiki/Duck_typing

Call the fit-predict on a newly instantiated `DummyClassifier` object, and find a way to extract the accuracy `score` from the test data. You may implement an accuracy function yourself or just use the `sklearn.metrics.accuracy_score` function. 

Finally, compare the accuracy score from your `DummyClassifier` with the scores found in [HOML] "Measuring Accuracy Using Cross-Validation", p.83. Are they comparable? 

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.metrics import accuracy_score

class DummyClassifier(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

#Fit dataset
dum_clf = DummyClassifier()
dum_clf.fit(X_train, y_train_5)

#Predict and print a 5
dum_pred5 = dum_clf.predict(X_test)
print('Prediction for X_test[15]: ',dum_pred5[15])
MNIST_PlotDigit(X_test[15])    

print('Accuracy score:', accuracy_score(y_test_5, dum_pred5))

Accuracy scoren ligger lavere end dem på s. 83 i HOML. Men de er dog stadig sammenlignelige. 

### Qd Conclusion

Now, conclude on all the exercise above. 

Write a short textual conclusion (max. 10- to 20-lines) that extract the _essence_ of the exercises: why did you think it was important to look at these particular ML concepts, and what was our overall learning outcome of the exercises (in broad terms).

Hvis man kigger på nøjagtigheden for de to classifiers, så er det nemt at se, at SGDClassifier har en bedre nøjagtighed end den selvimplementerede DummyClassifier. SGDClassifier har en nøjagtiged på 0.978, mens DummyClassifier kun har en nøjagtighed på 0.9108. Man skal dog her være opmærksom på, at denne nøjagtighed kan variere fra gang til gang, når man fitter sit datasæt. 
Man kan også se på de tal, som er blevet udvalgt, at SGDClassifier har gættet rigtig på X_test[15] (og korrekt klassificeret dette som et 5 tal), men at DummyClassifier ikke klassificeret dette korrekt. (Hvilket også giver god mening, da den altid giver negativ)

Læringen fra denne øvelse er, at nogen klassifier er bedre end andre til forskellige ting. Det kan derfor være en god idé at tjekke dem op imod hinanden, og finde den bedste på SIT datasæt. Dette kan dog godt tage længere tid, men som sæt af DummyClassifier og SGDClassifier, så er der forskel på nøjagtigheden, hvor man her vil vælge SGDClassifier over DummyClassifier. 